In [1]:
from src.datasets.CustomDataset import CustomDataset
import torch

training_data  = CustomDataset("data/encoded/train_sequences.npy", "data/encoded/train_targets.npy")
train_loader = torch.utils.data.DataLoader(training_data, batch_size=64, shuffle=True)
train_features, train_labels, attention_mask = next(iter(train_loader))
print(f"Feature batch shape: {train_features.shape}")
print(f"Labels batch shape: {train_labels.shape}")
print(f"Attention mask batch shape: {attention_mask.shape}")


val_data  = CustomDataset("data/encoded/val_sequences.npy", "data/encoded/val_targets.npy")
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=True)
val_features, val_labels, attention_mask= next(iter(val_loader))
print(f"Feature batch shape: {val_features.shape}")
print(f"Labels batch shape: {val_labels.shape}")
print(f"Attention mask batch shape: {attention_mask.shape}")

Feature batch shape: torch.Size([64, 512])
Labels batch shape: torch.Size([64])
Attention mask batch shape: torch.Size([64, 512])
Feature batch shape: torch.Size([64, 512])
Labels batch shape: torch.Size([64])
Attention mask batch shape: torch.Size([64, 512])


In [2]:
from src.models.EncoderClassifier import EncoderClassifier

import yaml

with open('/Users/johannes/Documents/Uni/transformer_bachelor/parameter.yaml', 'r') as file:
    config = yaml.safe_load(file)
    
# Model initialisieren
model = EncoderClassifier(
    vocab_size=config["data"]["vocab_size"],
    num_classes=config["data"]["num_classes"],
    embedding_dim=config["model"]["embedding_dim"],
    num_encoder_layers=config["model"]["num_encoder_layers"],
    num_heads=config["model"]["num_heads"],
    max_len=config["data"]["max_seq_length"],
    dropout_rate=config["model"]["dropout_rate"],
)
print(model)



EncoderClassifier(
  (embedding): Embedding(
    (token_embedding): Embedding(31, 512, padding_idx=0)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (classifier): Sequential(
    (

In [3]:
import torch.nn as nn
adam_opimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)
criterion = nn.CrossEntropyLoss() #wie funktionert die im Gegensatz zu Categorical Cross Entropy



In [4]:
from src.models.Trainer import Trainer

# Trainer initialisieren
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=adam_opimizer,
    criterion=criterion
)

# Training starten
trainer.train(
    num_epochs=100,
    save_path='best_model.pth',
    early_stopping_patience=3
)

# Metriken plotten
trainer.plot_metrics()


Epoch 1/100


Training:   0%|          | 0/1301 [00:00<?, ?it/s]

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
Embedding Layer
torch.Size([64, 512])


Training:   0%|          | 0/1301 [00:02<?, ?it/s]


KeyboardInterrupt: 